# Setup

In [ ]:
# Load environment variables
    # Dependencies:
    # - python-dotenv

from dotenv import load_dotenv, dotenv_values
import os

# Load env vars
    # Manual
    # load_dotenv("../.env/master.env")
    # maps_key = os.getenv("maps_key")

# Load all variables from the .env file into a dictionary
env_vars = dotenv_values("../.env/master.env")

# Set them as environment variables
for key, value in env_vars.items():
    os.environ[key] = value
    print(f"Loaded: {key}")

# Print
print(f"\nSuccessfully loaded {len(env_vars)} environment variables from master.env")

# print(env_vars.get("VAR1")) # call variable
# VAR1 = env_vars.get("VAR1")

Loaded: VAR1
Loaded: maps_key
Loaded: atlas_key_name
Loaded: atlas_key

Successfully loaded 4 environment variables from master.env


# Cleaning

### Load data

In [76]:
# Municipalities
    # Statistisches Bundesamt (2025)

import pandas as pd

# Load data
df_mun = pd.read_csv(
    '../data/processed/municipalities.csv',
    # Set data types
    dtype={
        'id': 'int64',
        'mun_key': str,
        'mun_name': str,
        'mun_name_short': str,
        'mun_zip': str,
        'mun_pop_cen22': 'int64',
        'mun_pop_cen22m': 'int64',
        'mun_pop_cen22f': 'int64',
        'mun_dens_cen22': 'int64',
        'mun_sizekm2': 'float64',
        'state': str,
        'lat': 'float64',
        'lon': 'float64',
        'geo': str
        }
    )

# Shorten municipality name
df_mun['mun_name_short'] = df_mun['mun_name'].str.split(',').str[0]
df_mun['mun_name_short'] = df_mun['mun_name_short'].astype(str)  # correct data type

# Print
print("\nMunicipalities data loaded successfully with correct data types.\n")
df_mun
df_mun.dtypes


Municipalities data loaded successfully with correct data types.



id                  int64
mun_key               str
mun_name              str
mun_name_short        str
mun_zip               str
mun_pop_cen22       int64
mun_pop_cen22m      int64
mun_pop_cen22f      int64
mun_dens_cen22      int64
mun_sizekm2       float64
state                 str
lat               float64
lon               float64
geo                   str
dtype: object

### Geocoding

In [ ]:
# Add coordinates
    # Dependencies:
    # - geopy
    # - Google Maps Geocoding API
    # Note: ZIP codes *alone* do not always resolve to postal codes or locations during geocoding, hence, mun_name

import pandas as pd
import time
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter

# Initialize geocoder
geolocator = GoogleV3(api_key=env_vars.get("maps_key"))
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.05) # 50 request/sec allowed

# Function to geocode each row
def get_coordinates(row):
    try:
        query = f"{row['mun_name_short']}, {row['mun_zip']}, Germany"
        location = geocode(query)
        if location:
            return pd.Series({'lat': location.latitude, 'lon': location.longitude})
        else:
            return pd.Series({'lat': None, 'lon': None})
    except Exception as e:
        print(f"Error geocoding {row['mun_name_short']}: {e}")
        return pd.Series({'lat': None, 'lon': None})

# Apply geocoding to each row
df_mun[['lat', 'lon']] = df_mun.apply(get_coordinates, axis=1)

# Combine into geo col (string: "lat,lon")
df_mun['geo'] = df_mun['lat'].astype(str) + ',' + df_mun['lon'].astype(str)

# Print
print("\nGeocoding completed. Updated municipalities data:\n")
df_mun.head()


Geocoding completed. Updated municipalities data:



,id,mun_key,mun_name,mun_name_short,mun_zip,mun_pop_cen22,mun_pop_cen22m,mun_pop_cen22f,mun_dens_cen22,mun_sizekm2,state,lat,lon,geo
0,1,110000000000,"Berlin, Stadt",Berlin,10178,3685265,1810259,1875006,4136,891.12,NaN,52.522188,13.409331,"52.5221879,13.4093313"
1,2,020000000000,"Hamburg, Freie und Hansestadt",Hamburg,20038,1862565,911888,950677,2467,755.09,NaN,53.548828,9.987170,"53.5488282,9.987170299999999"
2,3,091620000000,"München, Landeshauptstadt",München,80313,1505005,737442,767563,4844,310.70,NaN,48.135125,11.581981,"48.1351253,11.5819806"
3,4,053150000000,"Köln, Stadt",Köln,50667,1024621,498050,526571,2530,405.02,NaN,50.938644,6.953885,"50.9386437,6.9538847"
4,5,064120000000,"Frankfurt am Main, Stadt",Frankfurt am Main,60311,756021,371235,384786,3045,248.31,NaN,50.110090,8.682249,"50.1100897,8.6822492"


In [ ]:
# Write data
df_mun.to_csv('../data/processed/municipalities_geocoded.csv', index=False, encoding='utf-8')  # UTF-8 encoding

# Print
print("\nMunicipalities data (geocoded) saved successfully.")
print("\nRows:")
df_mun.shape[0]


Municipalities data (geocoded) saved successfully.


### Adding federal states

In [ ]:
# Add federal states
    # Dependencies:
    # - OpenPLZ API
    # Example: curl -X GET 'https://openplzapi.org/de/Localities?postalCode=13156' -H 'accept: text/json' | ConvertFrom-Json | ConvertTo-Json

# Potential corrections:
# - retries when timeout
# - exponential backoff for rate limiting (error status 429)

import requests
import json
import time

# Initialize/clear list for state names
states = []

# Rate limiting
REQUEST_DELAY = 0.5  # max 2 requests/sec

# Iterate over each postal code (zip) in df_mun
for i, postal_code in enumerate(df_mun['mun_zip']):
    try:
        # Build API URL with zip
        url = f'https://openplzapi.org/de/Localities?postalCode={postal_code}'
        response = requests.get(url)
        response.encoding = 'utf-8'  # Force UTF-8 encoding
        
        # Extract state name from first result
        data = response.json()
        if data and len(data) > 0:
            state_name = data[0]['federalState']['name']
            states.append(state_name)
        else:
            states.append(None)  # No data found for this postal code
        
        # Wait before next request
        if i < len(df_mun) - 1:  # No wait after the very last request
            time.sleep(REQUEST_DELAY)

    except Exception as e:
        states.append(None)  # Handle errors gracefully

# Add state column to df_mun
df_mun['state'] = states

print("\nAdding federal states: 1st run completed. Updated municipalities data:\n")
print(f"\nRows: {df_mun.shape[0]}\n")
df_mun.head()

# Unique values
state_counts = df_mun['state'].value_counts(dropna=False).reset_index()
state_counts.columns = ['State', 'Count']
state_counts = state_counts.sort_values('State')
state_counts

In [ ]:
# Write data
df_mun.to_csv('../data/processed/municipalities_geocoded_state1.csv', index=False, encoding='utf-8')  # UTF-8 encoding

# Print
print("\nMunicipalities data (geocoded + states [1st run] ) saved successfully.")

In [ ]:
# Clear list "states"
states = []

# Catch & re-run empty responses
    # Iterate over each NaN (state) in df_mun

for i, row in df_mun[df_mun['state'].isna()].iterrows():
    try:
        # Build API URL with mun_name_short
        url = f'https://openplzapi.org/de/FullTextSearch?searchTerm={row["mun_name_short"]}'
        response = requests.get(url)
        response.encoding = 'utf-8'  # Force UTF-8 encoding
        
        # Extract state name from first result
        data = response.json()
        if data and len(data) > 0:
            state_name = data[0]['federalState']['name']
            df_mun.at[i, 'state'] = state_name
        
        # Wait before next request
        time.sleep(REQUEST_DELAY)

    except Exception as e:
        pass  # Handle errors gracefully

# Print
print("\nAdding federal states: 2st (final) run completed. Updated municipalities data:\n")
print(f"\nRows: {df_mun.shape[0]}\n")

df_mun.head()

### Adding federal states (backup)

In [134]:
# Add federal states
    # Dependencies: -

import pandas as pd

# Load data: Municipalities (geocoded)

df_mun = pd.read_csv(
    '../data/processed/municipalities_geocoded.csv',
    # Set data types
    dtype={
        'id': 'int64',
        'mun_key': str,
        'mun_name': str,
        'mun_name_short': str,
        'mun_zip': str,
        'mun_pop_cen22': 'int64',
        'mun_pop_cen22m': 'int64',
        'mun_pop_cen22f': 'int64',
        'mun_dens_cen22': 'int64',
        'mun_sizekm2': 'float64',
        'state': str,
        'lat': 'float64',
        'lon': 'float64',
        'geo': str
        }
    )

# Load data: ZIP codes & Federal states

df_zip = pd.read_csv(
    '../data/processed/plz23.csv', 
    delimiter=';', 
    encoding='utf-8',
    dtype={
        'PLZ': 'str',
        'ORT': str,
        'ZUSATZ': 'str',
        'BUNDESLAND': str
    }
)

In [135]:
# Clean ZIP data

# Rename
df_zip = df_zip[['PLZ', 'ORT', 'BUNDESLAND']]
df_zip = df_zip.rename(columns={'PLZ': 'zip', 'ORT': 'municipality', 'BUNDESLAND': 'state'})

# Correct values for "Berlin"
df_zip.loc[df_zip['municipality'] == 'Berlin', 'state'] = 'Berlin'

# Drop NA
df_zip = df_zip.dropna(subset=['state'])

# Drop duplicates (for later matching)
df_zip = df_zip.drop_duplicates(subset=['zip'])

# Examine unique values
state_counts_zip = df_zip['state'].value_counts(dropna=False).reset_index()
state_counts_zip.columns = ['state', 'count']
state_counts_zip = state_counts_zip.sort_values('state')

# Print
print("\nUnique states in the dataset:\n")
print(state_counts_zip)
print("\nHeader:\n")
print(df_zip.head())


Unique states in the dataset:

                     state  count
1        Baden-Württemberg   1189
0                   Bayern   2073
11                  Berlin    191
9              Brandenburg    211
15                  Bremen     33
13                 Hamburg    100
5                   Hessen    551
12  Mecklenburg-Vorpommern    191
3            Niedersachsen    809
2      Nordrhein-Westfalen    876
4          Rheinland-Pfalz    643
14                Saarland     69
7                  Sachsen    378
10          Sachsen-Anhalt    205
6       Schleswig-Holstein    441
8                Thüringen    213

Header:

     zip municipality    state
0  01067      Dresden  Sachsen
1  01069      Dresden  Sachsen
2  01097      Dresden  Sachsen
3  01099      Dresden  Sachsen
4  01108      Dresden  Sachsen


In [ ]:
# Step 1: Match by zip code to fill state column
df_mun['state'] = df_mun['mun_zip'].map(df_zip.set_index('zip')['state']).fillna(df_mun['state'])

# Step 2: For unmatched rows, match by municipality name
mun_to_state = df_zip.groupby('municipality')['state'].first()
mask = df_mun['state'].isna()
df_mun.loc[mask, 'state'] = df_mun.loc[mask, 'mun_name_short'].map(mun_to_state)

# Check results
unmatched = df_mun['state'].isna().sum()
print(f"Rows still unmatched: {unmatched} out of {len(df_mun)}")

# Drop remaining unmatched rows
# -- not necessary --

# Examine unique values
state_counts = df_mun['state'].value_counts(dropna=False).reset_index()
state_counts.columns = ['state', 'count']
state_counts = state_counts.sort_values('state')
state_counts

# Print
print("\n----\nMatching complete.")
print("\nUnique states in the dataset:\n")  # city-states should be *unique* !
print(state_counts)
print("\nHeader:\n")
df_mun.head()

Rows still unmatched: 3 out of 2059

----
Matching complete.

Unique states in the dataset:

                     state  count
1        Baden-Württemberg    313
0                   Bayern    319
14                  Berlin      1
8              Brandenburg    112
16                  Bremen      1
15                 Hamburg      1
3                   Hessen    190
10  Mecklenburg-Vorpommern     85
5            Niedersachsen    160
2      Nordrhein-Westfalen    273
6          Rheinland-Pfalz    129
12                Saarland     17
4                  Sachsen    164
9           Sachsen-Anhalt    108
11      Schleswig-Holstein     63
7                Thüringen    120
13                     NaN      3

Header:



,id,mun_key,mun_name,mun_name_short,mun_zip,mun_pop_cen22,mun_pop_cen22m,mun_pop_cen22f,mun_dens_cen22,mun_sizekm2,state,lat,lon,geo
0,1,110000000000,"Berlin, Stadt",Berlin,10178,3685265,1810259,1875006,4136,891.12,Berlin,52.522188,13.409331,"52.5221879,13.4093313"
1,2,020000000000,"Hamburg, Freie und Hansestadt",Hamburg,20038,1862565,911888,950677,2467,755.09,Hamburg,53.548828,9.987170,"53.5488282,9.987170299999999"
2,3,091620000000,"München, Landeshauptstadt",München,80313,1505005,737442,767563,4844,310.70,Bayern,48.135125,11.581981,"48.1351253,11.5819806"
3,4,053150000000,"Köln, Stadt",Köln,50667,1024621,498050,526571,2530,405.02,Nordrhein-Westfalen,50.938644,6.953885,"50.9386437,6.9538847"
4,5,064120000000,"Frankfurt am Main, Stadt",Frankfurt am Main,60311,756021,371235,384786,3045,248.31,Hessen,50.110090,8.682249,"50.1100897,8.6822492"


In [137]:
# Write data
df_mun.to_csv('../data/processed/municipalities_full.csv', index=False, encoding='utf-8')  # UTF-8 encoding

# Print
print("\nMunicipalities data (geocoded + states) saved successfully.")


Municipalities data (geocoded + states) saved successfully.


# Sampling Origins: Municipalities

In [138]:
# Sample Origin locations

import pandas as pd

# Load data
df_mun = pd.read_csv(
    '../data/processed/municipalities_full.csv',
    # Set data types
    dtype={
        'id': 'int64',
        'mun_key': str,
        'mun_name': str,
        'mun_name_short': str,
        'mun_zip': str,
        'mun_pop_cen22': 'int64',
        'mun_pop_cen22m': 'int64',
        'mun_pop_cen22f': 'int64',
        'mun_dens_cen22': 'int64',
        'mun_sizekm2': 'float64',
        'state': str,
        'lat': 'float64',
        'lon': 'float64',
        'geo': str
        }
    )

print("\nMunicipalities data loaded successfully.")
#df_mun.head()


Municipalities data loaded successfully.


In [ ]:
# Function to get top, bottom, median municipalities for each state

def get_sample_municipalities(group):
    sorted_group = group.sort_values('mun_pop_cen22', ascending=False)
    
    # Only process if group has enough municipalities
    if len(sorted_group) < 9:
        # For small groups, label "all"
        result = sorted_group.copy()
        result['sample_type'] = 'all'
        return result
    
    # Get largest 3
    top = sorted_group.head(3).copy()
    top['sample_type'] = 'top'
    
    # Get smallest 3
    bottom = sorted_group.tail(3).copy()
    bottom['sample_type'] = 'bottom'
    
    # Get median 3
    n = len(sorted_group)
    mid_start = max(0, (n // 2) - 1)
    mid_end = min(n, mid_start + 3)
    median = sorted_group.iloc[mid_start:mid_end].copy()
    median['sample_type'] = 'median'
    
    return pd.concat([top, bottom, median]).drop_duplicates(subset=['mun_key'])

# Apply to each state and create new df
df_mun_sample = df_mun.groupby('state').apply(get_sample_municipalities).reset_index(level=0).reset_index(drop=True)

df_mun_sample = df_mun_sample[['mun_key', 'sample_type', 'mun_name_short', 'mun_pop_cen22', 'state', 'lat', 'lon']]

# Small sample
df_mun_sample_small = df_mun_sample[df_mun_sample['state'] == 'bayern']  # Selected Bayern (BY) for small sample (n=9)

# Print
print("\nSample municipalities selected successfully. Sample data:\n")
df_mun_sample


Sample municipalities selected successfully. Sample data:



,mun_key,sample_type,mun_name_short,mun_pop_cen22,state,lat,lon
0,081110000000,top,Stuttgart,612663,Baden-Württemberg,48.782703,9.182863
1,082120000000,top,Karlsruhe,309050,Baden-Württemberg,49.006750,8.393843
2,083110000000,top,Freiburg im Breisgau,237460,Baden-Württemberg,47.993854,7.846758
3,084375001047,bottom,Hettingen,1861,Baden-Württemberg,48.218306,9.233478
4,081255007103,bottom,Widdern,1782,Baden-Württemberg,49.316383,9.415555
...,...,...,...,...,...,...,...
115,160715053061,bottom,Neumark,462,Thüringen,51.079752,11.247166
116,160695004052,bottom,Ummerstadt,455,Thüringen,50.260149,10.812872
117,160690062062,median,Römhild,6383,Thüringen,50.385059,10.575825
118,160630103103,median,Werra-Suhl-Tal,6028,Thüringen,50.929777,10.112156


In [ ]:
# Write data
df_mun_sample.to_csv('../data/processed/municipalities_full_sample.csv', index=False, encoding='utf-8')  # UTF-8 encoding
# df_mun_sample_small.to_csv('../data/processed/municipalities_full_sample_small.csv', index=False, encoding='utf-8')

# Print
print("\nSampled municipalities data saved successfully.")


Sampled municipalities data saved successfully.


# Sampling Origins: Probes

In [ ]:
# Sampling RIPE Atlas Probe locations
    # Dependencies:
    # - RIPE Atlas API (atlas_key_name, atlas_key)

### Collect probe identifiers

In [ ]:
# Query the population (all healthy probes with coordinates and ipv4 address in Germany)

import requests
import pandas as pd

def get_all_probes(country_code="DE"):
    url = "https://atlas.ripe.net/api/v2/probes/"
    params = {
        "country_code": country_code,
        "fields": "id,asn_v4,geometry",  # collect these fields
        "tags": "system-ipv4-works",  # filter by system tag; system-ipv6-works, system-ipv4-stable-1d, system-ipv4-stable-30d, system-ipv4-stable-90d
        "status": 1,  # only active probes
        "format": "json",
        "page_size": 100
    }
    
    probes = []
    
    while url:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        for probe in data["results"]:
            coords = probe.get("geometry") or {}
            coordinates = coords.get("coordinates", [None, None])
            probes.append({
                "probe_id": probe["id"],
                "asn": probe.get("asn_v4"),
                "lon": coordinates[0],  # GeoJSON is [lon, lat]
                "lat": coordinates[1],
            })
        
        print(f"Fetched {len(probes)} probes so far...")
        url = data.get("next")
        params = {}
    
    df = pd.DataFrame(probes, columns=["probe_id", "asn", "lat", "lon"])
    df = df.dropna(subset=["lat", "lon"])  # exclude probes without coordinates
    df["asn"] = df["asn"].astype("Int64")  # convert to integer type
    
    return df

df_probes_de = get_all_probes("DE")

# Print
# expected: ~1759
print(f"\nTotal probes with coordinates: {len(df_probes_de)}\n")
# print(df.to_string(index=False))
df_probes_de.dtypes
df_probes_de.head()

Fetched 100 probes so far...
Fetched 200 probes so far...
Fetched 300 probes so far...
Fetched 400 probes so far...
Fetched 500 probes so far...
Fetched 600 probes so far...
Fetched 700 probes so far...
Fetched 800 probes so far...
Fetched 900 probes so far...
Fetched 1000 probes so far...
Fetched 1100 probes so far...
Fetched 1200 probes so far...
Fetched 1300 probes so far...
Fetched 1400 probes so far...
Fetched 1500 probes so far...
Fetched 1600 probes so far...
Fetched 1700 probes so far...
Fetched 1761 probes so far...

Total probes with coordinates: 1761



,probe_id,asn,lat,lon
0,46,3320,49.4905,10.9495
1,67,3209,52.3905,9.8105
2,71,50629,53.0875,8.8305
3,80,3320,52.3875,9.8115
4,81,3209,49.1475,9.2915


In [ ]:
# Write data

from datetime import datetime

# Generate timestamp
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')

# Write
df_probes_de.to_csv(f'../data/processed/probes_de_{timestamp}.csv', index=False, encoding='utf-8')  # UTF-8 encoding

# Print
print("\nProbes data (id, asn, health, ipv4, coordinates) saved successfully.")


Probes data (id, asn, health, ipv4, coordinates) saved successfully.


### Create distance matrix

In [ ]:
from geopy.distance import geodesic
import pandas as pd
import numpy as np

# Initialize distance matrix
# Rows: probe_id, Columns: mun_key
distance_data = {}

# Calculate distance for each probe to each municipality
for _, probe in df_probes_de.iterrows():
    probe_coords = (probe['lat'], probe['lon'])
    distances = []
    
    for _, mun in df_mun_sample.iterrows():
        mun_coords = (mun['lat'], mun['lon'])
        # Calculate geodesic distance in kilometers
        dist_km = geodesic(probe_coords, mun_coords).kilometers
        distances.append(dist_km)
    
    distance_data[probe['probe_id']] = distances

# Create DataFrame with probe_ids as columns, mun_keys as index
df_probes_distance = pd.DataFrame(
    distance_data,
    index=df_mun_sample['mun_key']
).T

# Set index name
df_probes_distance.index.name = 'probe_id'

print(f"\nDistance matrix created: {df_probes_distance.shape[0]} probes × {df_probes_distance.shape[1]} municipalities")
print(f"\nFirst 5 rows and columns:")
df_probes_distance.iloc[:5, :5]


Distance matrix created: 1761 probes × 120 municipalities

First few rows and columns:


mun_key,081110000000,082120000000,083110000000,084375001047,081255007103
probe_id,,,,,
46,151.040448,193.680033,282.409358,189.394100,113.001679
67,403.781905,389.475251,508.706577,465.906034,343.108953
71,479.533947,455.008818,570.873656,542.403095,421.528487
80,403.458065,389.171635,508.406591,465.579911,342.782016
81,41.341277,67.423397,166.802773,103.417930,20.842327


In [ ]:
# Write probe distance matrix to CSV
df_probes_distance.to_csv('../data/processed/probes_distance_matrix.csv', encoding='utf-8')

print("\nFull distance matrix saved successfully.")
print(f"Shape: {df_probes_distance.shape[0]} probes × {df_probes_distance.shape[1]} municipalities")


Full distance matrix saved to '../data/processed/probes_distance_matrix.csv'
Shape: 1761 probes × 120 municipalities


### Matching locations to closest probes

In [ ]:
# Retrieve lowest distances
    # 5 lowest values for each mun_key (i.e., closest probes)

import pandas as pd
import numpy as np

# For each municipality (row in transposed view), find 5 closest probes
closest_probes = {}

for mun_key in df_probes_distance.columns:
    # Get distances for this municipality across all probes
    distances = df_probes_distance[mun_key]
    
    # Get the 5 smallest distances and their probe_ids
    top5 = distances.nsmallest(5)
    
    closest_probes[mun_key] = {
        f'probe-{i+1}': top5.index[i] for i in range(5)
    } | {
        f'distance-{i+1}': top5.values[i] for i in range(5)
    }

# Create df
df_probes_sample = pd.DataFrame.from_dict(closest_probes, orient='index')
df_probes_sample.index.name = 'mun_key'

# Reorder columns to group probe IDs and distances together
probe_cols = [f'probe-{i}' for i in range(1, 6)]
dist_cols = [f'distance-{i}' for i in range(1, 6)]
df_probes_sample = df_probes_sample[probe_cols + dist_cols]

df_probes_sample.head(10)

,probe-1,probe-2,probe-3,probe-4,probe-5,distance-1,distance-2,distance-3,distance-4,distance-5
mun_key,,,,,,,,,,
081110000000,19869,22109,21574,19836,19837,0.219192,0.366082,0.396241,0.403501,0.403501
082120000000,11194,52831,21421,21441,896,0.350609,0.350778,0.399705,0.422199,0.450911
083110000000,1013945,11729,1004771,1010559,29215,0.660241,1.966321,3.476046,4.190831,6.377792
084375001047,52839,19884,51094,52995,11296,11.799138,15.042199,24.252101,24.252101,26.778651
081255007103,29222,18811,25329,81,29951,16.592880,16.720524,20.260771,20.842327,21.648458
081275004047,1010576,62217,20250,55713,18811,17.252451,21.091341,26.484352,30.365620,31.501102
081350020020,1506,1006858,1011407,53353,24357,22.270402,24.359167,24.929800,25.714412,28.041415
083260003003,51017,32134,1009411,33886,6761,13.553900,14.697697,18.883913,26.284859,30.270263
083370116116,4332,1005212,1013945,1004771,11729,10.849342,24.377096,41.289827,41.605536,42.624785


In [ ]:
# Write probe sampling to CSV
df_probes_sample.to_csv('../data/processed/probes_sample.csv', encoding='utf-8')

print("\nProbes sampling saved successfully.")


Probes sampling saved successfully.


In [ ]:
# Reduced Sampling for Bayern

# Load ../data/processed/probes_sample.csv as df_probes_sample
df_probes_sample = pd.read_csv('../data/processed/probes_sample.csv')

# Load ../data/processed/municipalities_full_sample.csv as 
df_mun_sample = pd.read_csv('../data/processed/municipalities_full_sample.csv')

# Select probes from df_probes_sample (whose mun_key belongs to state "Bayern" in df_mun_sample)
df_mun_keys_bayern = df_mun_sample[df_mun_sample['state'] == 'Bayern']['mun_key']
df_probes_bayern = df_probes_sample[df_probes_sample['mun_key'].isin(df_mun_keys_bayern)]

# Add mun_name_short by merging with df_mun_sample
df_probes_bayern = df_probes_bayern.merge(
    df_mun_sample[['mun_key', 'mun_name_short']], 
    on='mun_key', 
    how='left'
)

# Print list of probes (9*5=45)

# print(df_mun_keys_bayern.head(20))
print(df_probes_bayern.head(20))

       mun_key  probe-1  probe-2  probe-3  probe-4  probe-5  distance-1  \
0  91620000000  1010220  1010221  1000792    61562    32652    0.369909   
1  95640000000  1000661  1013659  1010379  1001538  1011139    0.286338   
2  97610000000  1002071    31056    22386    34752  1014021    0.504603   
3  94775437129      706    62428    50852    29659  1013075   14.131767   
4  94755428146    50852      706  1013075  1011248    29659    6.241085   
5  96775621181  1010374  1011093    52685    50445  1013840    6.035901   
6  97740121121    10920    53353     1506    51834    50828   10.680619   
7  92780118118  1007600  1008391  1003651  1007433  1007933   23.864524   
8  95730120120    60757    15128    34005       46    22981    9.539632   

   distance-2  distance-3  distance-4  distance-5 mun_name_short  
0    0.369909    0.376944    0.391089    0.418203        München  
1    0.390012    0.396487    0.396901    0.396901       Nürnberg  
2    0.662187    0.758538    1.382201   10.92485

In [11]:
# Write probe sample to continuous list

# Extract all probe IDs from the probe columns
probe_ids = []
for col in ['probe-1', 'probe-2', 'probe-3', 'probe-4', 'probe-5']:
    # probe_ids.extend(df_probes_sample[col].tolist())  # large sample
    probe_ids.extend(df_probes_bayern[col].tolist())  # small sample: Bayern

# Get unique probe IDs
unique_probe_ids = sorted(set(probe_ids))
print(f"\nTotal unique probes in the sample: {len(unique_probe_ids)}")
print("Head:", unique_probe_ids[:5])
print("Tail:", unique_probe_ids[-5:])

# Write unique IDs to text file
# with open('../data/processed/probes_sample_ids.txt', 'w') as f:  # large sample
with open('../data/processed/probes_bayern_ids.txt', 'w') as f:  # small sample: Bayern
    for probe_id in unique_probe_ids:
        f.write(f"{probe_id},")  # seperated by comma

print(f"\nSuccessfully written {len(unique_probe_ids)} unique probe IDs to text file.")
print("----")

# Check for multiple occurrences of each probe ID
# probe_id_counts = pd.Series(all_probe_ids).value_counts().sort_values(ascending=False)
# print("\nCount probe IDs (number of times each probe appears in the sample):\n")
# print("Head:\n", probe_id_counts.head(5))
# print("\nTail:\n", probe_id_counts.tail(5))


Total unique probes in the sample: 41
Head: [46, 706, 1506, 10920, 15128]
Tail: [1011248, 1013075, 1013659, 1013840, 1014021]

Successfully written 41 unique probe IDs to text file.
----


# Sampling Targets: Cloud Instances

### Set up Measurements

In [ ]:
# Sampling target machines for RIPE Atlas probes
    # Dependencies:
    # - ripe.atlas.cousteau
    # - RIPE Atlas API

In [ ]:
# Create a Measurement

from ripe.atlas.cousteau import Ping, AtlasSource, AtlasCreateRequest
from datetime import datetime, timezone, timedelta

ATLAS_API_KEY = env_vars.get("atlas_key")

# Origin:
probe_ids  = [1014162,1014259,1014434,1014441]

# Target:
hostnames = ["aws-dub.ncc.dock.ee"]  # more than one: will initiate multiple measurements

# Schedule:
CET = timezone(timedelta(hours=1))
start = datetime(2026, 2, 18, 10, 15, tzinfo=CET)  # YYYY, MM, DD, HH, MM
stop  = datetime(2026, 2, 18, 10, 45, tzinfo=CET)  # default: tzinfo=timezone.utc

# Only one Ping definition per hostname
measurements = [
    Ping(
        af=4, 
        target=hostname, 
        description=f"Ping {hostname}",  # af=6 for IPv6
        #tags=["test"],
        packets=3,
        size=48
        packet_interval=100,    # ms between packets, default: 0
        spread=30,              # spread probes over *observation interval* within *recurrence interval*
        resolve_on_probe=true,  # local DNS lookup
        skip_dns_check=false,   #
        include_probe_id=true,  # include probe ID in results for matching
        interval=300            # recurrence/frequency: e.g. every 5 min (omit if is_oneoff)
    )
    for hostname in hostnames
]

# One source with the explicit probe list
source = AtlasSource(
    type="probes",
    value=",".join(str(p) for p in probe_ids),
    requested=len(probe_ids)
)

atlas_request = AtlasCreateRequest(
    key=ATLAS_API_KEY,
    measurements=measurements,
    sources=[source],
    is_oneoff=true,   # set True to ignore interval and run once
    start_time=start,
    stop_time=stop,
)

is_success, response = atlas_request.create()
if is_success:
    print("Measurement IDs:", response.get("measurements"))
else:
    print("Error:", response)

Measurement IDs: [155921813]


### Augment data

In [ ]:
# Load data: Target Machine Specifics

import pandas as pd

df_machines = pd.read_excel(
    '../data/processed/zones_v2.xlsx', 
    engine='calamine',
    sheet_name='machines', 
    dtype={
        'asn': str,
        'ip4': str,
        'ip6': str,
        'hostname': str,
        'machine_type': str,
        'cost_h': 'float64',
        'cost_runtime': 'float64',
        'cpu': 'Int64',
        'ram': 'Int64',
        'netspeed': 'Int64'
    }
)

print("\nTarget machine data loaded successfully.")
# df_machines.head()

In [ ]:
# Load data: Zone Specifics

# Import "zones" from ../data/processed/zones_v2.xlsx
# to df_targets


# base sheet: "zones"

# add from "machines"
    # asn
    # ip4
    # ip6
    # host
    # machine_type
    # cost_h
    # cost_runtime
    # cpu
    # ram
    # netspeed

# add from "probes"
    # probe_id
    
    
# Retrieve coordinates for lat, long, geo

In [ ]:
# Calculate distance matrix: coordinates in df_targets to coordinates in df_mun_sample

### DNS

In [ ]:
# Load data: Target Machine Specifics

df_dns = pd.read_excel(
    '../data/processed/zones_v2.xlsx', 
    engine='calamine',
    sheet_name='dns', 
    dtype=str)  # Interpret all columns as string

# Drop col "asn"
df_dns = df_dns.drop(columns='asn')

print("\nMachine networking data loaded successfully.")
# df_dns.head()


Target machine data loaded successfully.


In [196]:
# Clean up

df_dns_clean = df_dns.dropna(subset=['probe_id'])  # keep only rows with probe_id

df_dns_clean = df_dns_clean[df_dns_clean['name'] != 'ovh-par']  # drop one entry due to incompatibility; manual setup

# df_dns_clean

In [ ]:
# Create BIND DNS records file from df_dns to create domain names at Cloudflare

# Dry-run
# df_dns_clean = df_dns_clean.head(1)

import pandas as pd

# Insert df (if none present)

# Define vars
zone = "dock.ee"   # your zone/domain
ttl  = 60          # TTL in seconds

lines = [
    f"$ORIGIN {zone}.",
    f"$TTL {ttl}",
    "",
]

for _, row in df_dns_clean.iterrows():
    hostname = row['hostname']
    # Make hostname relative to zone, or use @ for apex
    relative = hostname.replace(f".{zone}", "").replace(zone, "@")

    if pd.notna(row.get('ip4')) and row['ip4']:
        lines.append(f"{relative}\t{ttl}\tIN\tA\t{row['ip4']}")

    if pd.notna(row.get('ip6')) and row['ip6']:
        lines.append(f"{relative}\t{ttl}\tIN\tAAAA\t{row['ip6']}")

zone_file = "\n".join(lines)

output_path = "../data/processed/dns_cloudflare_bulk.txt"
with open(output_path, "w") as f:
    f.write(zone_file)

print("\nDNS Zone file successfully written.")
# print(zone_file)

In [ ]:
# Export Hostnames to file (newlines)

df_dns_lists = df_dns.dropna(subset=['probe_id'])  # keep only rows with probe_id

# Export hostnames and IPs to text file
with open('../data/processed/hostnames.txt', 'w') as f:
    for _, row in df_dns_lists.iterrows():
        f.write(f"{row['hostname']},{row['ip4']},{row['ip6']}\n")  # newline for each entry

In [227]:
# Export Hostnames to file (single line)

df_dns_lists = df_dns.dropna(subset=['probe_id'])  # keep only rows with probe_id

# Export hostnames and IPs to text file
with open('../data/processed/hostnames.txt', 'w') as f:
    values = []
    for _, row in df_dns_lists.iterrows():
        values.extend([str(row['hostname']), str(row['ip4']), str(row['ip6'])])  # all on a single line
    f.write(','.join(values))

# Archive

In [ ]:
# Example RIPE Atlas API
import requests
import json

probe_id = 1014464
response = requests.get(f"https://atlas.ripe.net/api/v2/probes/{probe_id}/")
data = response.json()

json_data = response.json()  # Parse JSON
print(json.dumps(json_data, indent=4)) # Print with indentation

coordinates = data['geometry']['coordinates']
longitude, latitude = coordinates

# Display as individual values
latitude, longitude

In [ ]:
# Example geopy using OSM

import pandas as pd
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode("Frankfurt am Main, 60311")

if location:
    print(f"Location: {location.address}")
    print(f"Latitude: {location.latitude}")
    print(f"Longitude: {location.longitude}")
else:
    print("Location not found")

In [ ]:
# Example Usage OpenPLZ API
# Dependencies: -

# Shell: curl -X GET 'https://openplzapi.org/de/FederalStates' -H 'accept: text/json' | ConvertFrom-Json | ConvertTo-Json

import requests
import json

# GET request to API
# List all federal states in Germany
response = requests.get('https://openplzapi.org/de/FederalStates')

# print(response.text)  # Print raw response
# print(response.json())  # Print raw JSON object
# response.json() # Print in native-style

json_data = response.json()  # Parse JSON
print(json.dumps(json_data, indent=4)) # Print with indentation

# Write JSON
# with open('output.json', 'w') as f:
#     json.dump(json_data, f, indent=4)